# SupermarQ

In [1]:
import supermarq
import cirq_superstaq
import cirq

## Setup
SupermarQ is built atop the `cirq_superstaq` framework. This means that most of the
details necessary for submitting circuits to various backends can be subsumed
by this package and SupermarQ can offer a simple, clean interface to the user.

### Service Initialization
The first thing we will need is a `cirq_superstaq.Service` which will allow us
to enter our Super.tech api keys as well as any IBM access tokens we may have
available separately.

In [2]:
superstaq_api_key = "ya29.a0ARrdaM8HavRWyhBsDOD4qbFqgHtts3FvQOVXlY8GserucshNNfb28eRFrpyrSxGsVpps7f51t5jX5EULVsycYqVglWTuBRWUpyt8Ew0Lt056WQgyatriJDwY7ZS5KoBtn6ZNtgLULoFFz9JHffb3OvelTJaEKA"
ibm_token = "70c3bf3be7eed9265577858ab058697602b7b831f742556d83594f3655fcd192de2160c2469f3432f1ac54263eb8eb38bf7b4d10519eaf2f420f72e0e3ec7f72"

In [8]:
service = cirq_superstaq.Service(
    api_key=superstaq_api_key,
    verbose=True,
)
service.ibmq_set_token(ibm_token)
print(service.get_balance())
print(service.get_backends())

$374.55
{'compile-only': ['aqt_keysight_qpu', 'sandia_qscout_qpu', 'neutral_atom_qpu', 'hqs_lt-s1-apival_qpu', 'hqs_lt-s1_simulator', 'hqs_lt-s1_qpu', 'hqs_lt-s2_qpu'], 'compile-and-run': ['ibmq_qasm_simulator', 'ibmq_armonk_qpu', 'ibmq_santiago_qpu', 'ibmq_bogota_qpu', 'ibmq_lima_qpu', 'ibmq_belem_qpu', 'ibmq_quito_qpu', 'ibmq_statevector_simulator', 'ibmq_mps_simulator', 'ibmq_extended-stabilizer_simulator', 'ibmq_stabilizer_simulator', 'ibmq_manila_qpu', 'ibmq_montreal_qpu', 'ibmq_toronto_qpu', 'ibmq_mumbai_qpu', 'ibmq_guadalupe_qpu', 'ibmq_brooklyn_qpu', 'ibmq_jakarta_qpu', 'ibmq_hanoi_qpu', 'ibmq_lagos_qpu', 'ibmq_cairo_qpu', 'ibmq_auckland_qpu', 'ibmq_perth_qpu', 'ibmq_washington_qpu', 'aws_sv1_simulator', 'aws_dm1_simulator', 'rigetti_aspen-m-1_qpu', 'oxford_lucy_qpu', 'aws_tn1_simulator', 'ionq_ion_qpu', 'd-wave_dw-2000q-6_qpu', 'd-wave_advantage-system4.1_qpu', 'aqt_keysight_qpu', 'sandia_qscout_qpu', 'neutral_atom_qpu', 'hqs_lt-s1-apival_qpu', 'hqs_lt-s1_simulator', 'hqs_lt-s

#### Testing that our Service is working properly

In [11]:
qubits = cirq.LineQubit.range(2)
circuit = cirq.Circuit(cirq.H(qubits[0]), cirq.CNOT(qubits[0], qubits[1]), cirq.measure(*qubits))
print(circuit)

job = service.create_job(circuit=circuit, repetitions=100, target="ibmq_santiago_qpu")
print("We created a job:", job.status())

0: ───H───@───M───
          │   │
1: ───────X───M───
We created a job: Queued


In [13]:
job.status()

'Queued'

## Benchmark creation

In [20]:
nq = 3
?supermarq.mermin_bell.MerminBell
merminbell = supermarq.benchmarks.mermin_bell.MerminBell(nq)

Init signature: supermarq.mermin_bell.MerminBell(num_qubits: int) -> None
Docstring:     
The Mermin-Bell benchmark is a test of a quantum computer's ability
to exploit purely quantum phenomemna such as superposition and entanglement.
It is based on the famous Bell-inequality tests of locality.

Performance is based on a QPU's ability to prepare a GHZ state and measure
the Mermin operator.
File:           ~/Workspace/SuperTech/SupermarQ/supermarq/benchmarks/mermin_bell.py
Type:           type
Subclasses:     


In [27]:
print(merminbell)
?merminbell.circuit

Signature: merminbell.circuit() -> cirq.circuits.circuit.Circuit
Docstring: Returns the quantum circuit corresponding to the current benchmark parameters.
File:      ~/Workspace/SuperTech/SupermarQ/supermarq/benchmarks/mermin_bell.py
Type:      method


In [28]:
merminbell_circuit = merminbell.circuit()
print(merminbell_circuit)

0: ───Rx(-0.5π)───@───────────@───────X───X───S───@───H───────M───
                  │           │       │   │       │           │
1: ───────────────X───@───H───┼───@───┼───@───────@───S───H───M───
                      │       │   │   │                       │
2: ───────────────────X───H───X───X───@───S───H───────────────M───


## Feature vector generation

In [25]:
?merminbell.score

Signature: merminbell.score(counts: collections.Counter) -> float
Docstring:
Compute the score for the N-qubit Mermin-Bell benchmark.

This function assumes the regular big endian ordering of bitstring results
File:      ~/Workspace/SuperTech/SupermarQ/supermarq/benchmarks/mermin_bell.py
Type:      method
